# 路径操作的高级配置
## OpenAPI 的 operationId

你可以在路径操作中通过参数 operation_id 设置要使用的 OpenAPI operationId。

务必确保每个操作路径的 operation_id 都是唯一的。



In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/", operation_id="some_specific_id_you_define")
async def read_items():
    return [{"item_id": "Foo"}]

## 使用 路径操作函数 的函数名作为 operationId

如果你想用你的 API 的函数名作为 operationId 的名字，你可以遍历一遍 API 的函数名，然后使用他们的 APIRoute.name 重写每个 路径操作 的 operation_id。

你应该在添加了所有 路径操作 之后执行此操作。

In [ ]:
from fastapi import FastAPI
from fastapi.routing import APIRoute

app = FastAPI()


@app.get("/items/")
async def read_items():
    return [{"item_id": "Foo"}]


def use_route_names_as_operation_ids(app: FastAPI) -> None:
    """
    Simplify operation IDs so that generated API clients have simpler function
    names.

    Should be called only after all routes have been added.
    """
    for route in app.routes:
        if isinstance(route, APIRoute):
            route.operation_id = route.name  # in this case, 'read_items'


use_route_names_as_operation_ids(app)

## 从 OpenAPI 中排除

使用参数 include_in_schema 并将其设置为 False ，来从生成的 OpenAPI 方案中排除一个 路径操作（这样一来，就从自动化文档系统中排除掉了）。

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/", include_in_schema=False)
async def read_items():
    return [{"item_id": "Foo"}]

## docstring 的高级描述

你可以限制 路径操作函数 的 docstring 中用于 OpenAPI 的行数。

添加一个 \f （一个「换页」的转义字符）可以使 FastAPI 在那一位置截断用于 OpenAPI 的输出。

剩余部分不会出现在文档中，但是其他工具（比如 Sphinx）可以使用剩余部分。

In [ ]:
from typing import Optional, Set

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


class Item(BaseModel):
    name: str
    description: Optional[str] = None
    price: float
    tax: Optional[float] = None
    tags: Set[str] = []


@app.post("/items/", response_model=Item, summary="Create an item")
async def create_item(item: Item):
    """
    Create an item with all the information:

    - **name**: each item must have a name
    - **description**: a long description
    - **price**: required
    - **tax**: if the item doesn't have tax, you can omit this
    - **tags**: a set of unique tag strings for this item
    \f
    :param item: User input.
    """
    return item

# 额外的状态码

FastAPI 默认使用 JSONResponse 返回一个响应，将你的 路径操作 中的返回内容放到该 JSONResponse 中。

FastAPI 会自动使用默认的状态码或者使用你在 路径操作 中设置的状态码。

## 额外的状态码
如果你想要返回主要状态码之外的状态码，你可以通过直接返回一个 Response 来实现，比如 JSONResponse，然后直接设置额外的状态码。

例如，假设你想有一个 路径操作 能够更新条目，并且更新成功时返回 200 「成功」 的 HTTP 状态码。

但是你也希望它能够接受新的条目。并且当这些条目不存在时，会自动创建并返回 201 「创建」的 HTTP 状态码。

要实现它，导入 JSONResponse，然后在其中直接返回你的内容，并将 status_code 设置为为你要的值。



In [ ]:
from typing import Optional

from fastapi import Body, FastAPI, status
from fastapi.responses import JSONResponse

app = FastAPI()

items = {"foo": {"name": "Fighters", "size": 6}, "bar": {"name": "Tenders", "size": 3}}


@app.put("/items/{item_id}")
async def upsert_item(
    item_id: str, name: Optional[str] = Body(None), size: Optional[int] = Body(None)
):
    if item_id in items:
        item = items[item_id]
        item["name"] = name
        item["size"] = size
        return item
    else:
        item = {"name": name, "size": size}
        items[item_id] = item
        return JSONResponse(status_code=status.HTTP_201_CREATED, content=item)

当你直接返回一个像上面例子中的 Response 对象时，它会直接返回。

FastAPI 不会用模型等对该响应进行序列化。

确保其中有你想要的数据，且返回的值为合法的 JSON（如果你使用 JSONResponse 的话）

## 在 Response 中使用 jsonable_encoder
由于 FastAPI 并未对你返回的 Response 做任何改变，你必须确保你已经准备好响应内容。

例如，如果不首先将 Pydantic 模型转换为 dict，并将所有数据类型（如 datetime、UUID 等）转换为兼容 JSON 的类型，则不能将其放入JSONResponse中。

对于这些情况，在将数据传递给响应之前，你可以使用 jsonable_encoder 来转换你的数据。

In [ ]:
from datetime import datetime
from typing import Optional

from fastapi import FastAPI
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
from pydantic import BaseModel


class Item(BaseModel):
    title: str
    timestamp: datetime
    description: Optional[str] = None


app = FastAPI()


@app.put("/items/{id}")
def update_item(id: str, item: Item):
    json_compatible_item_data = jsonable_encoder(item)
    return JSONResponse(content=json_compatible_item_data)

# 自定义响应 - HTML，流，文件和其他

FastAPI 默认会使用 JSONResponse 返回响应。

你可以通过直接返回 Response 来重载它，参见 直接返回响应。

但如果你直接返回 Response，返回数据不会自动转换，也不会自动生成文档（例如，在 HTTP 头 Content-Type 中包含特定的「媒体类型」作为生成的 OpenAPI 的一部分）。

你还可以在 路径操作装饰器 中声明你想用的 Response。

你从 路径操作函数 中返回的内容将被放在该 Response 中。

并且如果该 Response 有一个 JSON 媒体类型（application/json），比如使用 JSONResponse 或者 UJSONResponse 的时候，返回的数据将使用你在路径操作装饰器中声明的任何 Pydantic 的 response_model 自动转换（和过滤）。

## 使用 ORJSONResponse
例如，如果你需要压榨性能，你可以安装并使用 orjson 并将响应设置为 ORJSONResponse。

导入你想要使用的 Response 类（子类）然后在 路径操作装饰器 中声明它。

In [ ]:
from fastapi import FastAPI
from fastapi.responses import ORJSONResponse

app = FastAPI()


@app.get("/items/", response_class=ORJSONResponse)
async def read_items():
    return [{"item_id": "Foo"}]

参数 response_class 也会用来定义响应的「媒体类型」。

在这个例子中，HTTP 头的 Content-Type 会被设置成 application/json。

并且在 OpenAPI 文档中也会这样记录。

ORJSONResponse 目前只在 FastAPI 中可用，而在 Starlette 中不可用。

## HTML 响应

使用 HTMLResponse 来从 FastAPI 中直接返回一个 HTML 响应。

导入 HTMLResponse。
将 HTMLResponse 作为你的 路径操作 的 response_class 参数传入。

In [ ]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse

app = FastAPI()


@app.get("/items/", response_class=HTMLResponse)
async def read_items():
    return """
    <html>
        <head>
            <title>Some HTML in here</title>
        </head>
        <body>
            <h1>Look ma! HTML!</h1>
        </body>
    </html>
    """

### 返回一个 Response

In [ ]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse

app = FastAPI()


@app.get("/items/")
async def read_items():
    html_content = """
    <html>
        <head>
            <title>Some HTML in here</title>
        </head>
        <body>
            <h1>Look ma! HTML!</h1>
        </body>
    </html>
    """
    return HTMLResponse(content=html_content, status_code=200)

### OpenAPI 中的文档和重载 Response

如果你想要在函数内重载响应，但是同时在 OpenAPI 中文档化「媒体类型」，你可以使用 response_class 参数并返回一个 Response 对象。

接着 response_class 参数只会被用来文档化 OpenAPI 的 路径操作，你的 Response 用来返回响应。

### 直接返回 HTMLResponse

In [ ]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse

app = FastAPI()


def generate_html_response():
    html_content = """
    <html>
        <head>
            <title>Some HTML in here</title>
        </head>
        <body>
            <h1>Look ma! HTML!</h1>
        </body>
    </html>
    """
    return HTMLResponse(content=html_content, status_code=200)


@app.get("/items/", response_class=HTMLResponse)
async def read_items():
    return generate_html_response()

在这个例子中，函数 generate_html_response() 已经生成并返回 Response 对象而不是在 str 中返回 HTML。

通过返回函数 generate_html_response() 的调用结果，你已经返回一个重载 FastAPI 默认行为的 Response 对象，

但如果你在 response_class 中也传入了 HTMLResponse，FastAPI 会知道如何在 OpenAPI 和交互式文档中使用 text/html 将其文档化为 HTML。

## 可用响应
这里有一些可用的响应。

要记得你可以使用 Response 来返回任何其他东西，甚至创建一个自定义的子类。

[LOOK THIS](https://fastapi.tiangolo.com/zh/advanced/custom-response)

### 对类似文件的对象使用 StreamingResponse

如果您有类似文件的对象（例如，由 open() 返回的对象），则可以在 StreamingResponse 中将其返回。

包括许多与云存储，视频处理等交互的库。


In [ ]:
from fastapi import FastAPI
from fastapi.responses import StreamingResponse

some_file_path = "large-video-file.mp4"
app = FastAPI()


@app.get("/")
def main():
    def iterfile():  # (1)
        with open(some_file_path, mode="rb") as file_like:  # (2)
            yield from file_like  # (3)

    return StreamingResponse(iterfile(), media_type="video/mp4")

注意在这里，因为我们使用的是不支持 async 和 await 的标准 open()，我们使用普通的 def 声明了路径操作。